# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [ ]:
# All import statements needed for the project, for example:

from geopandas import GeoDataFrame
import pathlib
from requests.exceptions import ReadTimeout
import time
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame
import sqlalchemy as db
from sodapy import Socrata
import glob
import psycopg2
from sqlalchemy import create_engine,text
from sqlalchemy.orm import declarative_base
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, Polygon
import unittest
from datetime import date
from sqlalchemy.dialects.postgresql import insert


In [ ]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "xX3rCbSDM4vF0QEfgh09b2ZWW"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "IEOR4501-XL"

DB_USER = "postgres"
#DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_URL = f"postgresql://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

QUERY_DIR = pathlib.Path("queries")

In [ ]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [ ]:
#downloading data from small chunks
def download_nyc_csv_data (year:int, starttime:str, endtime:str, url:str, filename:str ) -> str:

    filepath = f'{DATA_DIR}/{filename}_{year}.csv'
    query=f"""
    select * 
    where created_date between {starttime} 
    and {endtime}
    """

    if not filename:
        print(f"Downloading {url} to {filename}...")
        client = Socrata( "data.cityofnewyork.us",
                  "xX3rCbSDM4vF0QEfgh09b2ZWW",
                  username="yirong263@gmail.com",
                  password="UTDYnmz*zn2u3g6",
                  timeout=60)
        max_retries = 5
        retry_wait = 10  # Initial wait time in seconds

        while max_retries > 0:
            try:    
            # Set initial parameters for the SoQL query
                limit = 1000000  # Example limit
                offset = 0  # Start at the beginning
                total_records = 100000000  # Example total number of records you wish to download
                current_record = 0
                while current_record < total_records:
                    # Adjust the query to include the limit and offset
                    results = client.get(f"{url}",query= query+ f" limit {limit} offset {offset}")
                    
                    # Convert to DataFrame and save to CSV
                    df = pd.DataFrame.from_records(results)
                    df.to_csv(f'{filepath}', index=False)
                    
                    # Update the offset and current_record count
                    offset += limit
                    current_record += len(results)

                    # Optional: Print progress
                    print(f'Downloaded {current_record} of {total_records}')
                break
            
            except ReadTimeout:
                # Wait before retrying
                time.sleep(retry_wait)

                # Reduce the number of retries left

                max_retries -= 1
                # Increase the wait time for the next retry
                
                retry_wait *= 2
        
        print(f"Done downloading {url} from {year}.")

    else:
        print(f"Reading from {filepath}...")


    return filepath

In [ ]:
#unit test
download_nyc_csv_data (2015,"2015-01-01T00:00:00.000","2015-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')

In [ ]:
def load_and_clean_zipcodes(zipcode_datafile: str) -> GeoDataFrame:
    """
    Load and clean NYC zipcode data from a shapefile.
    Args:
    zipcode_datafile (str): The file path to the shapefile.
    
    Returns:
    GeoDataFrame: Cleaned geospatial data frame of NYC zipcodes.
    """
    # Load the shapefile using GeoPandas
    gdf = gpd.read_file(zipcode_datafile)
    
    # Remove unnecessary columns from the dataframe and invalid data
    columns_to_drop = ['BLDGZIP', 'STATE', 'ST_FIPS', 'CTY_FIPS', 'URL', 'SHAPE_AREA', 'SHAPE_LEN']
    gdf_cleaned = gdf.drop(columns=columns_to_drop)
    gdf.drop_duplicates(subset='ZIPCODE', keep='first', inplace=True)
    gdf.dropna(inplace=True)
    gdf.drop_duplicates(inplace=True)

    # Rename columns for clarity
    gdf_cleaned = gdf_cleaned.rename(columns={'PO_NAME': 'City'})

    # Set the coordinate reference system to EPSG 4326
    gdf_cleaned = gdf_cleaned.to_crs(epsg=4326)

    # Normalize column names and change the datatype
    gdf_cleaned.columns = [column_name.lower().replace(' ', '_') for column_name in gdf_cleaned.columns]
    gdf_cleaned['zipcode']=gdf_cleaned['zipcode'].astype(float).astype(int)
    gdf_cleaned['population']=gdf_cleaned['population'].astype(float).astype(int)

    return gdf_cleaned


In [ ]:
#unit test
load_and_clean_zipcodes(ZIPCODE_DATA_FILE)

In [ ]:
def download_and_clean_311_data()-> GeoDataFrame:
    #data downloading
    download_nyc_csv_data(2015,"2015-01-01T00:00:00.000","2015-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2016,"2016-01-01T00:00:00.000","2016-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2017,"2017-01-01T00:00:00.000","2017-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2018,"2018-01-01T00:00:00.000","2018-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2019,"2019-01-01T00:00:00.000","2019-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2020,"2020-01-01T00:00:00.000","2020-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2021,"2021-01-01T00:00:00.000","2021-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2022,"2022-01-01T00:00:00.000","2022-12-31T23:59:59.999","erm2-nwe9",'nyc_311_data')
    download_nyc_csv_data(2023,"2023-01-01T00:00:00.000","2015-09-30T23:59:59.999","erm2-nwe9",'nyc_311_data')

    # After downloading all chunks
    csv_files = glob.glob('data/nyc_311_data_*.csv')
    
    # Remove unnecessary columns by keeping only the ones you need for each file
    dfs=[]
    for file in csv_files:
        df=pd.read_csv(file)
        columns_needed = ['unique_key', 'created_date', 'complaint_type','incident_zip','latitude', 'longitude']  # Replace with actual column names
        df = df[columns_needed]

        #eliminate duplicate
        df.drop_duplicates(inplace=True)

        # Remove invalid data points
        df.dropna(inplace=True) 

        # Normalize column names
        df.columns = [column_name.lower().replace(' ', '_') for column_name in df.columns]

        dfs.append(df) # processed df and append to a list

    # Concatenate all DataFrames into one
    NYC311_df = pd.concat(dfs,ignore_index=True)
    NYC311_df.drop_duplicates(inplace=True)# header duplicate elimination
    NYC311_df = NYC311_df.drop_duplicates(subset=['unique_key'])

    # Normalize Column Types
    # unique_key 
    NYC311_df['unique_key'] = NYC311_df['unique_key'].astype(int)

    # change name into 'id_NYC311'
    NYC311_df.rename(columns={'unique_key': 'id_NYC311'}, inplace=True)

    #incident zip
    NYC311_df.rename(columns={'incident_zip': 'zipcode'}, inplace=True)
    NYC311_df['zipcode']=NYC311_df['zipcode'].astype(float).astype(int)
    NYC311_df = NYC311_df[NYC311_df['zipcode'].apply(lambda x: str(x).isdigit() and len(str(x)) == 5)] 

    #created_date
    NYC311_df.rename(columns={'created_date': 'date'}, inplace=True)#rename "date"
    NYC311_df = NYC311_df.sort_values(by='date')# sorting by date
    NYC311_df['date'] = pd.to_datetime(NYC311_df['date']).dt.strftime('%Y-%m-%d')#change date format into yyyy-mm-dd
        
    # Assuming df is your existing DataFrame with latitude and longitude columns
    NYC311_df = gpd.GeoDataFrame(NYC311_df, geometry=gpd.points_from_xy(NYC311_df['longitude'], NYC311_df['latitude']))
    NYC311_df.set_crs(epsg=4326, inplace=True)

    #save the combined DataFrame to a new CSV file
    # NYC311_df.to_csv('data/nyc_311_data.csv', index=False)
    return NYC311_df
    

In [ ]:
#unit test
download_and_clean_311_data()

In [ ]:
def download_and_clean_tree_data() -> GeoDataFrame:
    download_nyc_csv_data(2015,"2015-01-01T00:00:00.000","2015-12-31T23:59:59.999","5rq2-4hqu",'tree')
    tree_df=pd.read_csv('data/tree_2015.csv')
    # Remove unnecessary columns by keeping only the ones you need
    columns_needed = ['created_at', 'tree_id', 'status','zipcode','health','spc_common', 'latitude', 'longitude']  # Replace with actual column names
    tree_df = tree_df[columns_needed]

    # Remove invalid data points
    # This is highly dependent on the context of your data, but as an example:
    tree_df.drop_duplicates(inplace=True)
    tree_df.dropna(inplace=True)  

    # Normalize column names
    tree_df.columns = [column_name.lower().replace(' ', '_') for column_name in tree_df.columns]
    #created_at
    tree_df.rename(columns={'created_at': 'date'}, inplace=True)
    tree_df['date'] = pd.to_datetime(tree_df['date']).dt.strftime('%Y-%m-%d')#change date format into yyyy-mm-dd

    #zipcode
    tree_df['zipcode'] = tree_df['zipcode'].astype(int)
    tree_df=tree_df.sort_values('date')

    tree_df = gpd.GeoDataFrame(tree_df, geometry=gpd.points_from_xy(tree_df['longitude'], tree_df['latitude']))
    tree_df.set_crs(epsg=4326, inplace=True)
    return tree_df


In [ ]:
#unit test
download_and_clean_tree_data()

In [ ]:

def load_and_clean_zillow_data()-> DataFrame:
    """
    Load and clean Zillow rent data from a CSV file, and transform it to a long format.
    
    Returns:
    DataFrame: Cleaned and transformed data frame of Zillow rent data.
    """
    # Load the CSV data using Pandas
    zillow_data_path = DATA_DIR / "zillow_rent_data.csv"
    df = pd.read_csv(zillow_data_path)
    
    # Keep only necessary columns
    columns_to_keep = ['RegionName', 'State', 'City', 'Metro', 'CountyName'] + [col for col in df.columns if '-' in col]
    df_cleaned = df[columns_to_keep]
    
    # Remove rows with a significant number of missing values
    df_cleaned = df_cleaned.dropna(thresh=len(df_cleaned.columns)/2, axis=0)

    # Filter for rows where the State is 'NY'
    df_cleaned = df_cleaned[df_cleaned['State'] == 'NY']
    
    # Rename columns for clarity
    df_cleaned = df_cleaned.rename(columns={'RegionName': 'ZipCode'})

    # Convert the data from wide format to long format
    # Melt the DataFrame to have Date and Rent as separate columns
    date_columns = [col for col in df_cleaned.columns if '-' in col]
    df_cleaned = df_cleaned.melt(id_vars=['ZipCode', 'State', 'City', 'Metro', 'CountyName'],
                                     value_vars=date_columns,
                                     var_name='Date',
                                     value_name='Rent')
    df_cleaned['ZipCode']=df_cleaned['ZipCode'].astype(float).astype(int)
    # Normalize column names
    df_cleaned.columns = [column_name.lower().replace(' ', '_') for column_name in df_cleaned.columns]
    
    return df_cleaned

In [ ]:
#unit test
load_and_clean_zillow_data()

In [ ]:
def load_all_data():
    """
    Load all necessary data for the project.

    Returns:
    Tuple containing:
        - GeoDataFrame of cleaned NYC zipcode data.
        - GeoDataFrame of cleaned NYC 311 data.
        - GeoDataFrame of cleaned tree data.
        - DataFrame of cleaned Zillow rent data.
    """
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = download_and_clean_311_data()
    geodf_tree_data = download_and_clean_tree_data()
    df_zillow_data = load_and_clean_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

## Part 2: Storing Data

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table
from geoalchemy2 import Geometry
from sqlalchemy.ext.declarative import declarative_base

class ZipcodeArea(Base):
    __tablename__ = 'zipcode_areas'  # Replace with your actual table name
    # Assuming 'zipcode' is a unique identifier for each row
    zipcode = Column(Integer, primary_key=True)
    city = Column(String)
    population = Column(Integer)
    area = Column(Float)
    county = Column(String)
    geometry = Column(Geometry('POLYGON', srid=4326))  # Adjust the geometry type if needed

class NYC311Complaints(Base):
    __tablename__ = 'nyc311_complaints'
    id_nyc = Column(Integer, primary_key=True)
    date = Column(Date)
    complaint_type = Column(String)
    zipcode = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)
    geometry = Column(Geometry(geometry_type='POINT', srid=4326))

class Tree(Base):
    __tablename__ = 'trees'
    tree_id = Column(Integer, primary_key=True)
    date = Column(Date)
    status = Column(String)
    zipcode = Column(Integer)
    health = Column(String)
    spc_common = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    geometry = Column(Geometry(geometry_type='POINT', srid=4326))

class Rent(Base):
    __tablename__ = 'rents'
    id=Column(Integer, primary_key=True)
    zipcode = Column(Integer)
    state = Column(String)
    city = Column(String)
    metro = Column(String)
    countyname = Column(String)
    date = Column(Date)
    rent = Column(Float)

engine = create_engine(DB_URL)
Base.metadata.create_all(engine)



In [ ]:
#unit test
class TestZipcodeArea(unittest.TestCase):

    def test_initialization(self):
        zipcode_area = ZipcodeArea(
            zipcode=12345,
            city='New York',
            population=8000000,
            area=300.5,
            county='New York County',
            geometry='POLYGON'  # Simplified for the example
        )
        self.assertEqual(zipcode_area.zipcode, 12345)
        self.assertEqual(zipcode_area.city, 'New York')
        complaint = NYC311Complaints(
                id_nyc=1,
                date=date(2021, 1, 1),
                complaint_type='Noise',
                zipcode=10001,
                latitude=40.7128,
                longitude=-74.0060
            )
        self.assertEqual(complaint.id_nyc, 1)
        self.assertEqual(complaint.complaint_type, 'Noise')
        tree = Tree(
                tree_id=1,
                date=date.today(),
                status='Good',
                zipcode=10001,
                health='Healthy',
                spc_common='Oak',
                latitude=40.7128,
                longitude=-74.0060
            )
        self.assertEqual(tree.tree_id, 1)
        self.assertEqual(tree.status, 'Good')
        self.assertEqual(tree.health, 'Healthy')
        rents = Rent(
            id=1,
            zipcode=10001,
            state='NY',
            city='New York',
            metro='New York Metro',
            countyname='New York County',
            date=date.today(),
            rent=2000.00
        )
        self.assertEqual(rents.city, 'New York')
        self.assertEqual(rents.rent, 2000.00)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)




### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

Add ZipCode Data to Database

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:

from shapely import wkt

geodf_zipcode_data['geometry'] = geodf_zipcode_data['geometry'].apply(lambda geom: geom.wkt)
for index, row in geodf_zipcode_data.iterrows():
    insert_stmt = insert(ZipcodeArea).values(
        zipcode=row['zipcode'],
        city=row['city'],
        population=row['population'],
        area=row['area'],
        county=row['county'],
        geometry=row['geometry']
    )
    on_conflict_stmt = insert_stmt.on_conflict_do_update(
        index_elements=['zipcode'],  # Unique constraint or column(s) causing conflict
        set_=dict(
            city=row['city'],
            population=row['population'],
            area=row['area'],
            county=row['county'],
            geometry=row['geometry']
        )
    )
    session.execute(on_conflict_stmt)


In [ ]:
session.commit()

Add NYC 311 Complaint Data to Database

In [ ]:
from sqlalchemy.dialects.postgresql import insert
# Convert geometry column to WKT format
geodf_311_data['geometry'] = geodf_311_data['geometry'].apply(lambda geom: geom.wkt)

def insert_batch(session, model, data, batch_size=200000):
    """
    Inserts data in batches to the database.

    Args:
    session (Session): The SQLAlchemy session for database operations.
    model (Type[Base]): The SQLAlchemy model class representing the database table.
    data (DataFrame): The DataFrame containing data to be inserted.
    batch_size (int): The size of each batch for insertion.
    """
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i+batch_size].to_dict(orient='records')

        for record in batch:
            stmt = insert(model).values(record)
            do_nothing_stmt = stmt.on_conflict_do_nothing(index_elements=['id_NYC311'])
            session.execute(do_nothing_stmt)

        session.commit()

# Insert data in batches
insert_batch(session, NYC311Complaints, geodf_311_data)


Add Tree Data to Database

In [ ]:
geodf_tree_data['geometry'] = geodf_tree_data['geometry'].apply(lambda geom: geom.wkt)

In [ ]:

geodf_tree_data['geometry'] = geodf_tree_data['geometry'].apply(lambda geom: geom.wkt)

for index, row in geodf_tree_data.iterrows():
    # Create a TreeData object for each row
    tree = Tree(
        tree_id=row['tree_id'],
        date=row['date'],
        status=row['status'],
        zipcode=row['zipcode'],
        health=row['health'],
        spc_common=row['spc_common'],
        latitude=row['latitude'],
        longitude=row['longitude'],
        geometry=row['geometry']
    )
    # Add each TreeData object to the session
    session.add(tree)

In [ ]:
session.commit()

Add Zillow Rent Data to Database

In [ ]:
# Loop through each row in the DataFrame for Zillow rent data
for index, row in df_zillow_data.iterrows():
    # Create a ZillowRent object for each row (assuming ZillowRent is the model class)
    rent = Rent(
        zipcode=row['zipcode'],
        state=row['state'],
        city=row['city'],
        metro=row['metro'],
        countyname=row['countyname'],
        date=row['date'],
        rent=row['rent']
    )
    # Add each ZillowRent object to the session
    session.add(rent)

In [ ]:
# Commit the session to save all added objects to the database
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:

# Define the directory where query files will be saved
# Creates the directory if it does not exist
QUERY_DIR = pathlib.Path("queries")
QUERY_DIR.mkdir(exist_ok=True)

# Define the SQL query for Query 1
# This query finds the number of 311 complaints per zip code 
# between 2022-10-01 and 2023-09-30 and orders them in descending order
QUERY_1 = """
SELECT zipcode, COUNT(*) AS complaint_count
FROM nyc311_complaints
WHERE date BETWEEN '2022-10-01' AND '2023-09-30'
GROUP BY zipcode
ORDER BY complaint_count DESC;
"""

# Function to write the SQL query to a file
def write_query_to_file(query, outfile):
    """
    Writes a SQL query to a file.

    Args:
    query (str): The SQL query string.
    outfile (pathlib.Path): The file path where the query will be saved.
    """
    with open(outfile, 'w') as file:
        file.write(query)

# File path for saving the query
QUERY_1_FILENAME = QUERY_DIR / "complaints_per_zipcode.sql"

# Execute the query and print the results
with engine.connect() as conn:
    result = conn.execute(text(QUERY_1))
    for row in result:
        print(row)

# Write the query to a file
write_query_to_file(QUERY_1, QUERY_1_FILENAME)


### Query 2

In [ ]:
# Query 2: Finding the top 10 zip codes with the most trees
# This query aims to identify which 10 zip codes have the highest number of trees

QUERY_2 = """
SELECT zipcode, COUNT(*) AS tree_count
FROM trees
GROUP BY zipcode
ORDER BY tree_count DESC
LIMIT 10;
"""

# File path for saving the query
QUERY_2_FILENAME = QUERY_DIR / "top_10_zipcodes_by_trees.sql"

# Execute the query and print the results
with engine.connect() as conn:
    result = conn.execute(text(QUERY_2))
    for row in result:
        print(row)

# Write the query to a file
write_query_to_file(QUERY_2, QUERY_2_FILENAME)


### Query 3

In [ ]:
# Query 3: Calculating average rent in the areas with the most trees for August 2023
# This query identifies the average rent by zip code for the top 10 zip codes with the most trees, 
# specifically for the month of August 2023

QUERY_3 = """
WITH TopTreeZipCodes AS (
    SELECT r.zipcode
    FROM trees
    JOIN rents r ON trees.zipcode = r.zipcode
    GROUP BY r.zipcode
    ORDER BY COUNT(*) DESC
    LIMIT 10
)
SELECT ttz.zipcode, ROUND(CAST(AVG(r.rent) AS numeric), 2) AS average_rent
FROM TopTreeZipCodes ttz
JOIN rents r ON ttz.zipcode = r.zipcode
WHERE r.date BETWEEN '2023-08-01' AND '2023-08-31'
GROUP BY ttz.zipcode
ORDER BY COUNT(*) DESC;
"""

# File path for saving the query
QUERY_3_FILENAME = QUERY_DIR / "average_rent_in_green_areas.sql"

# Execute the query and print the results
with engine.connect() as conn:
    result = conn.execute(text(QUERY_3))
    for row in result:
        print(row)

# Write the query to a file
write_query_to_file(QUERY_3, QUERY_3_FILENAME)


### Query 4

In [ ]:
# Query 4: Correlation between area's rent, tree count, and number of 311 complaints
# This query finds the 5 zip codes with the lowest and highest average rent for January 2023,
# along with the tree count and complaint count for each zip code

QUERY_4 = """
WITH RentRanking AS (
    SELECT zipcode, ROUND(CAST(AVG(r.rent) AS numeric), 2) AS average_rent,
    RANK() OVER (ORDER BY AVG(rent)) AS rent_rank
    FROM rents r
    WHERE date BETWEEN '2023-01-01' AND '2023-01-31'
    GROUP BY zipcode
),
MaxRank AS (
    SELECT MAX(rent_rank) AS max_rank FROM RentRanking
)
SELECT rr.zipcode, rr.average_rent, 
       (SELECT COUNT(*) FROM trees WHERE zipcode = rr.zipcode) AS tree_count,
       (SELECT COUNT(*) FROM nyc311_complaints WHERE zipcode = rr.zipcode AND date BETWEEN '2023-01-01' AND '2023-01-31') AS complaint_count
FROM RentRanking rr, MaxRank
WHERE rr.rent_rank <= 5 OR rr.rent_rank >= (MaxRank.max_rank - 4)
ORDER BY rr.average_rent;
"""

# File path for saving the query
QUERY_4_FILENAME = QUERY_DIR / "rent_tree_complaint_correlation.sql"

# Execute the query and print the results
with engine.connect() as conn:
    result = conn.execute(text(QUERY_4))
    for row in result:
        print(row)

# Write the query to a file
write_query_to_file(QUERY_4, QUERY_4_FILENAME)


### Query 5

In [ ]:
# Query 5: Identifying areas with the most greenery using spatial join
# This query rewrites Query 2 to include a spatial join between the trees table and the zipcodes table
# to determine which trees are located within the boundary of a zipcode

QUERY_5 = """
WITH TreeCount AS (
    SELECT z.zipcode, COUNT(t.tree_id) AS tree_count
    FROM zipcode_areas z
    JOIN trees t ON ST_Within(t.geometry, z.geometry)
    GROUP BY z.zipcode
)
SELECT zipcode, tree_count
FROM TreeCount
ORDER BY tree_count DESC
LIMIT 10;
"""

# File path for saving the query
QUERY_5_FILENAME = QUERY_DIR / "greenery_areas_with_spatial_join.sql"

# Execute the query and print the results
with engine.connect() as conn:
    result = conn.execute(text(QUERY_5))
    for row in result:
        print(row)

# Write the query to a file
write_query_to_file(QUERY_5, QUERY_5_FILENAME)


### Query 6

In [ ]:
# Query 6: Finding trees within a ½ mile radius of a specific coordinate point
# This query identifies which trees are within a ½ mile radius of the given latitude and longitude

QUERY_6 = """
SELECT 
    tree_id, 
    spc_common AS species, 
    health, 
    status, 
    ST_AsText(geometry) AS geom
FROM 
    trees
WHERE 
    ST_DWithin(
        geometry,
        ST_SetSRID(ST_MakePoint(-73.96253174434912, 40.80737875669467), 4326),
        0.5 * 1609.34 -- 0.5 miles in meters

    );
"""


# File path for saving the query
QUERY_6_FILENAME = QUERY_DIR / "trees_nearby_coordinate.sql"

# Execute the query and print the results
with engine.connect() as conn:
    result = conn.execute(text(QUERY_6))
    for row in result:
        print(row)

# Write the query to a file
write_query_to_file(QUERY_6, QUERY_6_FILENAME)


## Part 4: Visualizing the Data

### Visualization 1

First, find the top 3 complaint types for October 1st, 2022 to September 30th, 2023 (inclusive). 

Then, create an appropriate visualization for the number of complaints per day over $timeframe for those complaint types. 

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    """
    Queries the database for data needed for visual 1 and returns it as a DataFrame.

    Returns:
    pd.DataFrame: DataFrame containing the queried data.
    """
    query="""
WITH TopComplaints AS (
    SELECT complaint_type
    FROM nyc311_complaints
    WHERE date BETWEEN '2022-10-01' AND '2023-09-30'
    GROUP BY complaint_type
    ORDER BY COUNT(*) DESC
    LIMIT 3
)
SELECT n.date, tc.complaint_type, COUNT(*) AS daily_complaints
FROM TopComplaints tc
JOIN nyc311_complaints n ON tc.complaint_type = n.complaint_type
WHERE n.date BETWEEN '2022-10-01' AND '2023-09-30'
GROUP BY n.date, tc.complaint_type
ORDER BY n.date, daily_complaints DESC;
    
"""

    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    df = pd.read_sql_query(query, engine)
    return df

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 300


def plot_visual_1(df: pd.DataFrame) -> None:
    """
    Plots a line chart showing daily complaints for the top 3 complaint types.

    Args:
    df (pd.DataFrame): The DataFrame containing the complaint data.
    """
    fig, ax = plt.subplots(figsize=(15, 8))

    top_types = df['complaint_type'].value_counts().nlargest(3).index
    df_top = df[df['complaint_type'].isin(top_types)]

    # Create a dummy plot to initialize the legend correctly
    for ctype in top_types:
        ax.plot([], [], label=ctype)

    def animate(i):
        ax.clear()
        sns.lineplot(data=df_top.iloc[:i], x='date', y='daily_complaints', hue='complaint_type', ax=ax)
        plt.title('Daily Complaints for Top 3 Complaint Types (Oct 1, 2022 - Sep 30, 2023)')
        plt.xlabel('Date')
        plt.ylabel('Number of Complaints')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()

    ax.legend(title='Complaint Types')  # Set the legend
    anim =animation.FuncAnimation(fig, animate, frames=len(df_top), interval=200)
    return HTML(anim.to_jshtml())#animation


In [ ]:
visualization1_dataframe = get_data_for_visual_1()
plot_visual_1(visualization1_dataframe)

### Visualization 2

Create a visualization that shows the number of complaints by complaint type for the top 10 complaints in zip code 10027 for October 1st, 2018 to September 30th, 2023 (inclusive). 


In [ ]:
def plot_visual_2(df)-> None :
    plt.figure(figsize=(15, 8))
    sns.barplot(data=df, x='complaint_type', y='complaint_count')
    plt.title('Top 10 Complaint Types in Zip Code 10027 (Oct 1, 2018 - Sep 30, 2023)')
    plt.xlabel('Complaint Type')
    plt.ylabel('Number of Complaints')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
def get_data_for_visual_2()-> pd.DataFrame:
    # SQL query to fetch top 10 complaint types in zip code 10027
    """
    Plots a bar chart showing the count of different complaint types.

    Args:
    df (pd.DataFrame): The DataFrame containing the complaint data.
    """
    query = """
    WITH TopComplaints AS (
        SELECT complaint_type, COUNT(*) AS count
        FROM nyc311_complaints
        WHERE zipcode = 10027 AND date BETWEEN '2018-10-01' AND '2023-09-30'
        GROUP BY complaint_type
        ORDER BY count DESC
        LIMIT 10
    )
    SELECT nyc311_complaints.complaint_type, COUNT(*) AS complaint_count
    FROM TopComplaints
    JOIN nyc311_complaints ON TopComplaints.complaint_type = nyc311_complaints.complaint_type
    WHERE nyc311_complaints.zipcode = 10027 AND nyc311_complaints.date BETWEEN '2018-10-01' AND '2023-09-30'
    GROUP BY nyc311_complaints.complaint_type
    ORDER BY complaint_count DESC;
    """

    # Execute the query and return a DataFrame
    df = pd.read_sql_query(query, engine)
    return df

In [ ]:
visualization2_dataframe = get_data_for_visual_2()
plot_visual_2(visualization2_dataframe)

### Visualization 3


Between January 1st, 2015 and September 30th, 2023 (inclusive), create a visualization using 2 subplots that share the x-axis where one subplot shows rent compared to the number of trees by zip code, and the other subplot shows rent compared to the number of complaints by zip code.


In [ ]:
# use a more descriptive name for your function
def plot_visual_3(df)-> None :
    """
    Plots scatter plots to compare average rent with number of trees and number of complaints.

    Args:
    df (pd.DataFrame): The DataFrame containing the data for average rent, tree count, 
                       and complaint count.
    """
    # Create a subgraph
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

    # Subgraph of rent versus number of trees
    ax1.scatter(df['average_rent'], df['tree_count'], alpha=0.6)
    ax1.set_ylabel('Number of Trees')
    ax1.set_title('Average Rent vs Number of Trees by Zip Code')

    # Subgraph of rent versus number of complaints
    ax2.scatter(df['average_rent'], df['complaint_count'], alpha=0.6, color='orange')
    ax2.set_xlabel('Average Rent ($)')
    ax2.set_ylabel('Number of Complaints')
    ax2.set_title('Average Rent vs Number of Complaints by Zip Code')

    # show chart
    plt.tight_layout()
    plt.show()

In [ ]:
def get_data_for_visual_3()-> pd.DataFrame:
    """
    Queries the database for data needed for visual 3 and returns it as a DataFrame.

    Returns:
    pd.DataFrame: DataFrame containing the queried data for rent, tree count, and complaint count.
    """
    # Query your database for the data needed.
    
    query="""
WITH RentData AS (
        SELECT zipcode, AVG(rent) AS average_rent
        FROM rents
        WHERE date BETWEEN '2015-01-01' AND '2023-09-30'
        GROUP BY zipcode
    ),
    TreeData AS (
        SELECT zipcode, COUNT(*) AS tree_count
        FROM trees
        GROUP BY zipcode
    ),
    ComplaintData AS (
        SELECT zipcode, COUNT(*) AS complaint_count
        FROM nyc311_complaints
        WHERE date BETWEEN '2015-01-01' AND '2023-09-30'
        GROUP BY zipcode
    )
    SELECT r.zipcode, r.average_rent, t.tree_count, c.complaint_count
    FROM RentData r
    LEFT JOIN TreeData t ON r.zipcode = t.zipcode
    LEFT JOIN ComplaintData c ON r.zipcode = c.zipcode;
    """

    df = pd.read_sql_query(query, engine)
    return df

In [ ]:
visualization3_dataframe = get_data_for_visual_3()
plot_visual_3(visualization3_dataframe)

### Visualization 4


Create a boxplot, where the x-axis is average rent in September 2023, separated into $1000 bins (i.e. $0-1000, $1001-2000, etc), and the y-axis is the number of 311 complaints observed in each zip code between October 1, 2022 (inclusive) to September 30, 2023 (inclusive).


In [ ]:
def plot_visual_4(df)-> None:
    """
    Plots a boxplot showing the number of 311 complaints by different rent bins.

    This function takes a DataFrame with rent bins and complaint counts and plots a boxplot
    to visualize the distribution of 311 complaints across different rent levels.

    Args:
        df (pd.DataFrame): The DataFrame containing the rent bin and complaint count data.
    """
    # Create the boxplot
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='rent_bin', y='complaint_count', data=df)
    
    # Customize the plot
    plt.title('Boxplot of 311 Complaints by Rent Bins (Sep 2023)')
    plt.xlabel('Average Rent Bins ($1000 increments)')
    plt.ylabel('Number of 311 Complaints')

    # Show the plot
    plt.show()

In [ ]:
def get_data_for_visual_4()-> pd.DataFrame:
    """
    Plots a boxplot showing the number of 311 complaints by different rent bins.

    Args:
    df (pd.DataFrame): The DataFrame containing the rent bin and complaint count data.
    """
    # Define SQL query to fetch required data
    query = """
    WITH RentBins AS (
        SELECT zipcode, 
               AVG(rent) AS average_rent,
               WIDTH_BUCKET(AVG(rent), 0, 10000, 10) AS rent_bin
        FROM rents
        WHERE date BETWEEN '2023-09-01' AND '2023-09-30'
        GROUP BY zipcode
    ),
    ComplaintsCount AS (
        SELECT zipcode, 
               COUNT(*) AS complaint_count
        FROM nyc311_complaints
        WHERE date BETWEEN '2022-10-01' AND '2023-09-30'
        GROUP BY zipcode
    )
    SELECT R.zipcode, 
           R.rent_bin, 
           R.average_rent, 
           C.complaint_count
    FROM RentBins R
    JOIN ComplaintsCount C ON R.zipcode = C.zipcode
    ORDER BY R.rent_bin;
    """

    # Execute the query and return a DataFrame
    df = pd.read_sql_query(query, engine)
    return df

In [ ]:
visualization4_dataframe = get_data_for_visual_4()
plot_visual_4(visualization4_dataframe)

### Visualization 5

Create a geospatial plot of the coordinates of reported 311 incidents that happened between January 1st, 2023 and September 30th, 2023 (inclusive) within a 1 kilometer radius of the same coordinate from Query 6 in Part 3.


In [ ]:
def plot_visual_5(df)-> None:
    """
    Plots a geospatial map showing recent 311 incidents in a specific area.

    Args:
    df (gpd.GeoDataFrame): The GeoDataFrame containing the geospatial data for plotting.
    """
    # Create a geospatial map
    fig, ax = plt.subplots(figsize=(10, 6))
    df.plot(ax=ax, marker='o', color='blue', markersize=5)
    
    # Center point (for visualization center)
    center_point = Point(-73.96253174434912, 40.80737875669467)
    plt.scatter([center_point.x], [center_point.y], color='red')  # 红点代表中心点

    # Set the chart title and label
    plt.title('Recent 311 Incidents in the Immediate Area (Jan 1, 2023 - Sep 30, 2023)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    # Show charts
    plt.show()

In [ ]:
def get_data_for_visual_5()-> gpd.GeoDataFrame:
     """
    Query and return geospatial data for NYC 311 complaints within a 1km radius of a specified point.

    This function retrieves latitude and longitude data for NYC 311 complaints that are within a 1000-meter radius
    of a given geographical point (latitude 40.80737875669467, longitude -73.96253174434912). 
    The data is filtered to include complaints registered between 2023-01-01 and 2023-09-30.

    Args:
        engine (Engine): A SQLAlchemy Engine object used to execute the database query.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing the queried data with geographical coordinates.
    """
    query = """
    SELECT latitude, longitude
FROM nyc311_complaints
WHERE ST_DWithin(
    geography(ST_MakePoint(longitude, latitude)),
    geography(ST_MakePoint(-73.96253174434912, 40.80737875669467)), 
    1000 -- distance in meters
)
AND date BETWEEN '2023-01-01' AND '2023-09-30';

    """
    
    df_query_result = pd.read_sql_query(query, engine)
    df = gpd.GeoDataFrame(
        df_query_result,
        geometry=gpd.points_from_xy(df_query_result.longitude, df_query_result.latitude)
    )
    return df

In [ ]:
visualization5_dataframe = get_data_for_visual_5()
plot_visual_5(visualization5_dataframe)

### Visualization 6

Create a geospatial plot of two sets of data: the coordinates of trees in NYC, and the coordinates of "New Tree Request" 311 complaint types that were made from October 1st, 2018 to September 30th, 2023 (inclusive). 

In [ ]:
def get_data_for_visual_6()-> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """
    Retrieve geospatial data for trees and new tree requests in NYC.

    This function performs two queries:
    1. It retrieves latitude and longitude data for all trees in NYC from the 'trees' table.
    2. It retrieves latitude and longitude data for 'New Tree Request' complaints registered 
       between 2018-10-01 and 2023-09-30 from the 'nyc311_complaints' table.

    Args:
        engine (Engine): A SQLAlchemy Engine object used to execute the database queries.

    Returns:
        Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame]: 
            - A GeoDataFrame containing the latitude and longitude of trees in NYC.
            - A GeoDataFrame containing the latitude and longitude of new tree requests in NYC.
    """
    query_trees = """
    SELECT latitude, longitude
    FROM trees;
    """

    query_new_tree_requests = """
    SELECT latitude, longitude
    FROM nyc311_complaints
    WHERE complaint_type = 'New Tree Request'
    AND date BETWEEN '2018-10-01' AND '2023-09-30';
    """

    df_trees = gpd.GeoDataFrame(
        pd.read_sql_query(query_trees, engine),
        geometry=gpd.points_from_xy(pd.read_sql_query(query_trees, engine).longitude, pd.read_sql_query(query_trees, engine).latitude)
    )
    df_new_tree_requests = gpd.GeoDataFrame(
        pd.read_sql_query(query_new_tree_requests, engine),
        geometry=gpd.points_from_xy(pd.read_sql_query(query_new_tree_requests, engine).longitude, pd.read_sql_query(query_new_tree_requests, engine).latitude)
    )

    return df_trees, df_new_tree_requests



In [ ]:
def plot_visual_6(df_trees, df_new_tree_requests)-> None:
    """
    Plot a geospatial chart showing locations of existing trees and new tree requests in NYC.

    This function takes two GeoDataFrames: one representing the location of existing trees,
    and the other representing new tree requests. It plots these locations on a map with
    distinct markers and colors for easy differentiation.

    Args:
        df_trees (gpd.GeoDataFrame): A GeoDataFrame containing the latitude and longitude of trees in NYC.
        df_new_tree_requests (gpd.GeoDataFrame): A GeoDataFrame containing the latitude and longitude of new tree requests in NYC.

    Returns:
        None: This function does not return anything but displays a matplotlib plot.
    """    
    # Create a geospatial chart
    fig, ax = plt.subplots(figsize=(10, 10))

    #Draw the new tree request location on the map
    df_new_tree_requests.plot(ax=ax, markersize=5, color='red', label='New Tree Requests')
    
    # Plot the tree position on the map
    df_trees.plot(ax=ax, markersize=5, color='green', label='Trees')

    # Customize the chart
    plt.title('Trees and New Tree Requests in NYC')
    plt.legend()

    # show chart
    plt.show()

In [ ]:
df_trees, df_new_tree_requests = get_data_for_visual_6()
plot_visual_6(df_trees, df_new_tree_requests)